In [12]:
import tensorflow as tf
import numpy as np

In [2]:
corpus_raw = "He is the king . The king is royal . She is the royal queen"

In [6]:
raw_sentence = corpus_raw.split(".")
sentences = []
for sentence in raw_sentence:
    sentences.append(sentence.strip().split())

sentences

[['He', 'is', 'the', 'king'],
 ['The', 'king', 'is', 'royal'],
 ['She', 'is', 'the', 'royal', 'queen']]

In [7]:
data = []
WINDOW_SIZE = 2

for sentence in sentences:
    for word_index, word in enumerate(sentence):
        start_index = max(word_index - WINDOW_SIZE, 0)
        end_index = min(word_index + WINDOW_SIZE + 1, len(sentence))

        for nb_word in sentence[start_index:word_index]:
            data.append([word, nb_word])

        for nb_word in sentence[word_index + 1 : end_index]:
            data.append([word, nb_word])


In [8]:
data

[['He', 'is'],
 ['He', 'the'],
 ['is', 'He'],
 ['is', 'the'],
 ['is', 'king'],
 ['the', 'He'],
 ['the', 'is'],
 ['the', 'king'],
 ['king', 'is'],
 ['king', 'the'],
 ['The', 'king'],
 ['The', 'is'],
 ['king', 'The'],
 ['king', 'is'],
 ['king', 'royal'],
 ['is', 'The'],
 ['is', 'king'],
 ['is', 'royal'],
 ['royal', 'king'],
 ['royal', 'is'],
 ['She', 'is'],
 ['She', 'the'],
 ['is', 'She'],
 ['is', 'the'],
 ['is', 'royal'],
 ['the', 'She'],
 ['the', 'is'],
 ['the', 'royal'],
 ['the', 'queen'],
 ['royal', 'is'],
 ['royal', 'the'],
 ['royal', 'queen'],
 ['queen', 'the'],
 ['queen', 'royal']]

In [9]:
words = []
for word in corpus_raw.split():
    if word != ".":
        words.append(word)
words = set(words)

word2int = {}
int2word = {}
vocab_size = len(words)

for i, word in enumerate(words):
    word2int[word] = i
    int2word[i] = word

print(word2int)
print(int2word)

{'She': 0, 'He': 1, 'king': 2, 'is': 3, 'queen': 4, 'the': 5, 'royal': 6, 'The': 7}
{0: 'She', 1: 'He', 2: 'king', 3: 'is', 4: 'queen', 5: 'the', 6: 'royal', 7: 'The'}


In [10]:
def to_one_hot(word_index, vocab_size):
    temp = np.zeros(vocab_size)
    temp[word_index] = 1
    return temp

In [13]:
x_train = []
y_train = []
for words in data:
    x_train.append(to_one_hot(word2int[words[0]], vocab_size))
    y_train.append(to_one_hot(word2int[words[1]], vocab_size))

In [15]:
x_train[:3]

[array([0., 1., 0., 0., 0., 0., 0., 0.]),
 array([0., 1., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 1., 0., 0., 0., 0.])]

In [16]:
y_train[:3]

[array([0., 0., 0., 1., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 1., 0., 0.]),
 array([0., 1., 0., 0., 0., 0., 0., 0.])]

In [17]:
x_train = np.asarray(x_train, dtype=np.float32)
y_train = np.asarray(y_train, dtype=np.float32)

In [18]:
x_train[:3]

array([[0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.]], dtype=float32)

In [27]:
class Word2Vec:
    def __init__(self, vocab_size=10, embedding_dim=5, optimizer='sgd',
                 epochs=1000, learning_rate=0.01):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.epochs = epochs
        if optimizer == 'adam':
            self.optimizer = tf.optimizers.Adam(learning_rate=learning_rate)
        else:
            self.optimizer = tf.optimizers.SGD(learning_rate=learning_rate)
        

        self.W1 = tf.Variable(tf.random.normal([self.vocab_size, 
                                                self.embedding_dim]))
        self.b1 = tf.Variable(tf.random.normal([self.embedding_dim]))

        self.W2 = tf.Variable(tf.random.normal([self.embedding_dim,
                                                self.vocab_size]))
        self.b2 = tf.Variable(tf.random.normal([self.vocab_size]))

    def vectorized(self, word_index):
        return (self.W1 + self.b1)[word_index]


    def train(self, x_train, y_train):
        for i in range(self.epochs):
            with tf.GradientTape() as tape:
                hidden_layer = tf.add(tf.matmul(x_train, self.W1), self.b1)
                output_layer = tf.add(tf.matmul(hidden_layer, self.W2), self.b2)

                pred = tf.nn.softmax(output_layer)
                loss = tf.reduce_mean(-tf.math.reduce_sum(y_train*tf.math.log(pred), axis=[1]))

                params = [self.W1, self.b1, self.W2, self.b2]
                grads = tape.gradient(loss, params)
                self.optimizer.apply_gradients(zip(grads,params))
            if i % 1000 == 0:
                print(loss)


In [28]:
w2v = Word2Vec(vocab_size=vocab_size, embedding_dim=5, optimizer='SGD', epochs=10000, learning_rate=0.1)
w2v.train(x_train, y_train)

tf.Tensor(3.7162118, shape=(), dtype=float32)
tf.Tensor(1.3957883, shape=(), dtype=float32)
tf.Tensor(1.3800758, shape=(), dtype=float32)
tf.Tensor(1.3773054, shape=(), dtype=float32)
tf.Tensor(1.3762228, shape=(), dtype=float32)
tf.Tensor(1.3756589, shape=(), dtype=float32)
tf.Tensor(1.3753175, shape=(), dtype=float32)
tf.Tensor(1.3750902, shape=(), dtype=float32)
tf.Tensor(1.374929, shape=(), dtype=float32)
tf.Tensor(1.3748091, shape=(), dtype=float32)


## Gensim

In [1]:
import pandas as pd
df_news = pd.read_csv('./data/news.csv')

In [2]:
df_news.head()

,date,media,title,content,ngrams
0,2005-01-01,연합인포맥스,내년 美국채수익률 5% 넘어서기 어려울 듯,2005년 10년만기 미국 국채수익률이 연방준비제도이사회(FRB)의 금리인상 지속...,"만기/NNG,국채/NNG,수익률/NNG,fed/NNG,fed/NNG,금리/NNG,인..."
1,2005-01-01,연합인포맥스,[뉴욕채권-마감] 10년만기 국채수익률 작년보다 낮은 수준서 마쳐,2년만기 국채가격 4년래 최악의 한해 보내 10년만기 미국 국채수익률이 ...,"만기/NNG,국채/NNG,가격/NNG,최악/NNG,보내/VV,만기/NNG,국채/NN..."
2,2005-01-01,연합인포맥스,[뉴욕환시] `내년초 달러-엔에 주력'..달러 對엔 하락,2004년 마지막 거래일인 31일 뉴욕환시에서 미국 달러화는 개장초 102엔 근처...,"마지막/NNG,거래일/NNG,뉴욕/NNG,환시/NNG,달러/NNG,개장/NNG,근처..."
3,2005-01-01,연합인포맥스,[31일 뉴욕금융시장 요약] 한산한 거래속 새해 준비,) 한해 마지막 날인 31일 뉴욕 주요 금융시장은 한산한 거래속에 새해를 준비하는 ...,"마지막/NNG,뉴욕/NNG,금융시장/NNG,한산/NNG,거래/NNG,새해/NNG,준..."
4,2005-01-02,연합인포맥스,美 증시 기술주 주도로 2년 연속 상승..'01년래 최고,지난해 뉴욕증시는 기술주 주도로 2년 연속연초 대비 상승하면서 대표지수들을 지난 ...,"주도/NNG,연속/NNG,대비/NNG,상승/NNG,대표지수/NNG,최고/NNG,오르..."


In [3]:
df_news['content'][30000]

'26일 오전 서울채권시장 국고채금리는 전주 미국채시장의 약세 영향으로 소폭 올라 출발한 후 재료 부족으로 정체됐다.    이날 국고5년 7-1 지표물 금리는 전주에서 1bp 오른 4.80%에서 개장한 후 4.81%로 올랐다.    전주 미국채 금리는 지난 2월 기존 주택판매가 예상 밖의 호조를 보여 상승했다.10년 만기 국채금리는 2bp 오른 연 4.61%에, 통화정책에 민감한 2년 만기 금리는 1bp 이상 높은 연 4.61%에 마감됐다.    연합인포맥스가 지난 19일부터 23일까지 17개 국내외 금융기관의 이코노미스트 들을 대상으로 실시한 폴에 따르면 2월 산업생산은 작년 같은 달에 비해 3.2%  증가할 것으로 집계됐다. 이 같은 컨센서스는 전월 증가율 7.4%보다 4.2%p 낮은 것으로 1월과 2월을 평균할 때는 5.3%를 보였다.    또 인포맥스가 같은 기간 국내 14개 경제연구소와 금융기관을 대상으로 설문조 사를 실시한 결과, 올해 1분기 GDP 증가율은 전년동기대비 3.87%로 예상됐다.    아울러 인포맥스가 같은 기간 14개 국내외 금융기관 및 경제연구소 이코노미스 트들을 대상으로 설문조사를 실시한 결과 3월 CPI는 전년동월대비 2.2% 상승할 것으로 추정됐다.    또 각종 입찰이 있었다.    한국은행은 2일 만기 5조원 규모 RP 매각 입찰에서 6조8천억원이  몰려,  5조원 전액이 연 4.57%에 낙찰됐다고 발표했다.    재정경제부는 국고20년 3천770억원 입찰에서 8천140억원이 응찰을 해와 전액이 연 5.14% 낙찰됐다고 밝혔다.    은행권 채권딜러는 "국고20년물이 시장 예상치 보다 1bp 정도 낮게 낙찰돼 시장분위기가 좀 강해질 수 있다"며 "하지만 장중 금리가 반락할 것 같지는 않다"고  예상했다.    이 딜러는 "국고20년 금리가 그 동안 다른 짧은 기간물에 비해 많이 올라서  상대적으로 입찰 때 매수가 강했다"며 "또 단기금리가 탄탄한 하방경직성을 보여서 금리 하락시도를 하기가 버겁다"고 덧붙였다.    libe

In [4]:
sentences = df_news['content'].apply(lambda x : x.split('.    ')).tolist()

In [6]:
sentences[0]

['2005년 10년만기 미국 국채수익률이  연방준비제도이사회(FRB)의 금리인상 지속에 따른 인플레이션 압력 완화로 연 5%를 넘어서기 어려울 것으로 전망됐다',
 '31일 씨티그룹 애널리스트들은 2005년에 미국의 인플레가 잘 제어될 것이라면서반면 내년 2년만기 국채수익률은 FRB의 지속적인 금리인상으로 연 4.00-4.50%  수준까지 상승하게 될 것이라고 예측했다',
 '씨티그룹은 단기 국채수익률이 상승세를 나타낼 것으로 보이는 반면 장기  국채수익률의 상승폭은 제한될 것으로 보여 수익률 곡선 평탄화가 가속화될 것이라고 덧붙였다',
 '씨티그룹은 내년 고용창출 호조가 가구당 수입증가를 견인할 것이라면서 고용시장 호전이 소비자지출을 떠받치게 될 것이라고 말했다']

In [8]:
len(sentences)

261817

In [16]:
from konlpy.tag import Mecab
mecab = Mecab()

In [17]:
corpus = [mecab.morphs(sent) for para in sentences for sent in para]

In [21]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=corpus, 
                 vector_size=100, 
                 alpha=0.025, 
                 window=5, 
                 min_count=1,
                 workers=4)

KeyboardInterrupt: 

In [15]:
word_embedding = model.wv['금리']

KeyError: "Key '금리' not present"